In [1]:
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch

In [23]:
#reset the environment
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "Kasemrat-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [24]:
id = {'virtual' : '270329335' ,'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409'}
detecters = {'QA' : ['156261350#0'], 'QB' : ['-453962283#3'], 'QC' : ['459551209#0', '459551209#3'], 'QD' : ['27702347#0', '27702347#4', '27702347#6']}

In [25]:
import sumolib
import lxml.etree as ET
def get_edge_id(POI):
#     root = ET.Element('additionals')
#     net = sumolib.net.readNet('map_grid_lock_add2TL_copy_newTLS.net.xml')
#     output = []
#     for key in POI:
#         edgeID = net.getEdge(key)
#         numOfLane = edgeID.getLaneNumber()
#         for lane_index in range(numOfLane):
#             output.append(key+'_'+str(lane_index))
#     print(output)
    if POI == ['156261350#0']:
        output = ['156261350#0_0']
    if POI == ['-453962283#3']:
        output = ['-453962283#3_0']
    if POI == ['459551209#0', '459551209#3']:
        output = ['459551209#0_0', '459551209#0_1', '459551209#0_2', '459551209#0_3', '459551209#3_0', '459551209#3_1', '459551209#3_2', '459551209#3_3']
    if POI == ['27702347#0', '27702347#4', '27702347#6']:
        output = ['27702347#0_0', '27702347#4_0', '27702347#6_0']
    return output    

In [26]:
for key in ['QA', 'QB', 'QC', 'QD']:
    get_edge_id(detecters[key])

In [27]:
def get_state(): 
    Length = []
    for key in ['QA', 'QB', 'QC', 'QD']:
        remain_length = 1-(sum([traci.lanearea.getJamLengthMeters(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))/traci.lanearea.getLength(get_edge_id(detecters[key])[0])
        Length.append(remain_length*100)
    occupancy = []
    for key in ['QA', 'QB', 'QC', 'QD']:
         occupancy.append(sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))
    occupancy+= Length
    current_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
    for i in current_phase:
        current_phase_binary = [0, 0, 0]
        current_phase_binary[int(i//2)] = 1
        occupancy+= current_phase_binary
    return occupancy

In [57]:
junction_name = ['virtual', 'KasemRat']
def take_action(action, ID):
    if action != None:
        w = 0.999
        act = [5, 1, 3]
        print(act[action])
        change_to_phase = (act[action]+1)%6
        if change_to_phase == traci.trafficlight.getPhase(id[junction_name[ID]]):
            traci.trafficlight.setPhase(id[junction_name[ID]], change_to_phase) # NO YELLOW PHASE 
        else:
            traci.trafficlight.setPhase(id[junction_name[ID]], act[action]) # YELLOW PHASE FIRST
    for i in range(5):
        traci.simulationStep()
#     w1 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])
#     w2 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])
#     reward = -w2*(1-w) - w1*(w)
    reward = sum(get_state()[4:8])
    print('reward', reward)
    return reward

In [58]:
def initial(state, congest_phase, threshold_max, threshold_min): 
    ##FLAG = 1 = Yellow FLAG = 0 = GREEN
    print(state)
    Length = state[4:8]
    occupancy = state[0:4]
    s1 = state[8:11] #virtual phase
    s2 = state[11:14] #Kasem phase
    # Virtual
    if congest_phase == None:
        phase_index = None
        if ((Length[0] <= threshold_max) or (Length[3] <= threshold_max)):
            congest_phase = 2
        elif (Length[2] <= threshold_max):
            congest_phase = 0
    if congest_phase == 2:
        phase_index = 1
        if ((occupancy[0] <= threshold_min) or (occupancy[3] <= threshold_min)):
            congest_phase = None
        elif (Length[2] <= threshold_max):
            congest_phase = 0         
    if congest_phase == 0:
        phase_index = 0
        if (occupancy[2] <= threshold_min):
            congest_phase = None
        elif ((Length[0] <= threshold_max) or (Length[3] <= threshold_max)):
            congest_phase = 2
    return phase_index, congest_phase

In [59]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [60]:
traci.close()

In [61]:
def episode0(threshold_max, threshold_min,volume_virtualJ):
    #episode = 0
    congest_phase = None
    reset()
    FLAG0 = 1
    FLAG2 = 1
    traci.simulationStep()
    for seconds in range(2880):
        state = get_state()
        phase_index, congest_phase = initial(state, congest_phase, threshold_max, threshold_min)
        reward = take_action(phase_index, 0)
        reward_memory.append(reward)
         
    plot_durations()
    traci.close()

In [ ]:
episode0(20, 20, 1000) #PEAM